# Exercise:  Multiple Agents

1) Try evolutionary algorithm for an optimization of your interest while varying the methods and parameters for mutaion, crossover, and selection.

2) Model some kind of interaction between people in a game theoretic framework, define payoff matrices, and identify Nash equilibria and ESSs.

3) See how the replicator dynamics of the rock-scissors-paper game changes with the parameter $a$.